In [ ]:
import os
from PIL import Image
import cv2
import numpy as np
from pathlib import Path
from lxml import etree
import os, sys, shutil
from matplotlib import pyplot as plt
import cv2
import numpy as np
from pathlib import Path
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import json

In [ ]:
data_path = Path(f'./razmetka')
out_foler = Path(f'./outpath')

for cls_folder in os.listdir(data_path):
    os.makedirs(out_foler / cls_folder, exist_ok=True)

    for filename in tqdm(os.listdir(data_path / cls_folder / 'images')):
        if '.jpg' not in filename:
            continue
        img = cv2.imread(str(data_path / cls_folder / 'images' / filename))
        mask = cv2.imread(str(data_path / cls_folder / 'masks' / filename.replace('.jpg', '.png')), cv2.IMREAD_GRAYSCALE)

        colors = np.unique(mask)
        for color in colors:
            if color == 0:
                continue
            swan_mask = (mask == color).astype(np.uint8)
            swan = cv2.bitwise_and(img, img, mask=swan_mask)
            cv2.imwrite(str(out_foler / cls_folder / filename), swan)

In [6]:
swan_mask

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [ True,  True,  True, ..., False, False, False],
       [ True,  True,  True, ..., False, False, False],
       [ True,  True,  True, ..., False, False, False]])

In [ ]:
!cat config.json

In [3]:
def get_polygons(img):
    colors = np.unique(img)
    polygons = []
    for color in colors:
        if color == 0:
            continue
        contours, hierarchy = cv2.findContours((img == color).astype(np.uint8), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        contours = list(sorted(contours, key=lambda c: cv2.contourArea(c)))
        polygons.append(contours[-1][:, 0].tolist())
    return polygons

In [4]:
data_path = Path(f'./datasets/yolo_swan_with_classify')
os.makedirs(data_path / 'images/train', exist_ok=True)
os.makedirs(data_path / 'images/val', exist_ok=True)
os.makedirs(data_path / 'labels/train', exist_ok=True)
os.makedirs(data_path / 'labels/val', exist_ok=True)

In [5]:
path_to_images = Path('razmetka')

In [6]:
all_paths = [path_to_images / cls / 'masks' / filename for cls in os.listdir(path_to_images) for filename in os.listdir(path_to_images / cls / 'masks')]
train, val = train_test_split(all_paths, test_size=0.05, random_state=42)

In [7]:
s = set()
is_segmentation = True
for suffix, paths in [['train', train], ['val', val]]:
    for path in tqdm(map(str, paths), total=len(paths)):
        label = 0 if 'шипун' in path else 1 if 'klikun' in path else 2
        s.add(label)

        yolo_annots = []

        name = path.split('/')[-1]
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        h_img, w_img = img.shape
        
        for polygon in get_polygons(img):
            figure = [label]

            if is_segmentation:
                for i, point in enumerate(polygon):
                    figure.extend([point[0] / w_img, point[1] / h_img])
                figure.extend(figure[1:3])
            else:
                xs, ys = list(map(lambda p: p[0], polygon)), list(map(lambda p: p[1], polygon))
                xc, yc = (max(xs) - min(xs)) / w_img, (max(ys) - min(ys)) / h_img
                w, h = (max(xs) / 2 + min(xs)) / w_img, (max(ys) / 2 + min(ys)) / h_img
                figure.extend([xc, yc, w, h])

            yolo_annots.append(figure)
 
        img_path = path.replace('/masks/', '/images/').replace('.png', '.jpg')
        out_img_path = str(data_path / 'images' / suffix / name)
        if os.path.exists(out_img_path):
            out_img_path = out_img_path.replace('.', '__2.')
        shutil.copy(img_path, out_img_path)

        yolo_annots = '\n'.join([' '.join(map(str, yolo_annot)) for yolo_annot in yolo_annots])
        with open(data_path / 'labels' / suffix / (name.split('.')[0] + '.txt'), 'w') as f:
            f.write(yolo_annots)

100%|██████████| 452/452 [01:21<00:00,  5.55it/s]


In [8]:
txt = '''
path: D:/vlad/min_nature/data/datasets/yolo_swan_with_classify  # dataset root dir
train: images/train  # train images (relative to 'path') 128 images
val: images/val  # val images (relative to 'path') 128 images
#test:  # test images (optional)

# Classes
names:
  0: mute
  1: whooper
  2: bewick
'''
with open('seg_swan_min_nature_with_classify.yaml', 'w') as f:
    f.write(txt)

In [2]:
txt = '''
path: D:/vlad/min_nature/data/datasets/yolo  # dataset root dir
train: images/train  # train images (relative to 'path') 128 images
val: images/val  # val images (relative to 'path') 128 images
#test:  # test images (optional)

# Classes
names:
  0: swan
'''
with open('seg_swan_min_nature.yaml', 'w') as f:
    f.write(txt)

In [ ]:
###yolo segment train data=seg_swan_min_nature_with_classify.yaml model=yolov8l-seg.yaml epochs=48 imgsz=640 patience=6 batch=24 save_period=2 workers=8 project=swan name=train_min_nature_with_classify verbose=False seed=42 single_cls=False nbs=72 device=0

In [ ]:
####yolo segment train data=seg_swan_min_nature.yaml model=yolov8l-seg.yaml epochs=80 imgsz=640 patience=6 batch=24 save_period=2 workers=8 project=swan name=train_min_nature verbose=False seed=42 single_cls=True nbs=72 device=0

In [ ]:
!yolo segment train data=data.yaml model=yolov8l-seg.yaml epochs=12 imgsz=640 patience=6 batch=32 save_period=1 workers=8 project=swan_beak name=train_custom verbose=False seed=42 single_cls=True nbs=32 device=0

In [ ]:
yolo segment train data=seg_swan_min_nature.yaml model=yolov8l-seg.yaml epochs=80 imgsz=640 patience=8 batch=28 save_period=2 workers=8 project=swan name=train_min_nature verbose=False seed=42 single_cls=True nbs=84 device=0

In [2]:
from ultralytics import YOLO
from copy import deepcopy

In [112]:
model = YOLO("swan/train_min_nature_with_classify/weights/epoch28.pt")

In [13]:
model = YOLO("swan/train_min_nature9/weights/epoch48.pt")

In [3]:
model = YOLO("swan/train_beak_min_nature_with_classify4/weights/epoch46.pt")

In [4]:
model.to('cuda')

In [15]:
data_path = Path(f'./razmetka')
out_foler = Path(f'./yolo_preds_swan_no_classify')
for cls_folder in os.listdir(data_path):
    os.makedirs(out_foler / cls_folder, exist_ok=True)
    for filename in tqdm(os.listdir(data_path / cls_folder / 'images')):
        if os.path.exists(str(out_foler / cls_folder / filename)):
            continue

        img = Image.open(str(data_path / cls_folder / 'images' / filename))
        results = model(img, verbose=False)

        img = cv2.cvtColor(np.asarray(img), cv2.COLOR_BGR2RGB)
        if img is None:
            continue

        cls = results[0].probs
        if results[0].masks is None:
            continue
        masks = results[0].masks.xy
        for swan_contours in masks:
            swan_mask = np.zeros_like(img)
            cv2.drawContours(swan_mask, [swan_contours.astype(np.int32)], 0, (255,255,255), -1)

            swan_mask = swan_mask[:, :, 0]
            contours, hierarchy = cv2.findContours(swan_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

            left = min([min(cnt[:, 0, 0]) for cnt in contours])
            right = max([max(cnt[:, 0, 0]) for cnt in contours])

            top = min([min(cnt[:, 0, 1]) for cnt in contours])
            bottom = max([max(cnt[:, 0, 1]) for cnt in contours])

            swan = deepcopy(img)[top:bottom, left:right]
            swan_mask = swan_mask[top:bottom, left:right]
            
            swan = cv2.bitwise_and(swan, swan, mask=swan_mask)
            cv2.imwrite(str(out_foler / cls_folder.replace('разметка_малый', 'malyi').replace('разметка_шипун', 'shipun') / filename), swan)

100%|█████████▉| 3011/3012 [05:48<00:00,  8.64it/s]


PermissionError: [Errno 13] Permission denied: 'razmetka\\разметка_шипун\\images\\masks'

In [117]:
data_path = Path(f'./razmetka')
out_foler = Path(f'./yolo_preds')
for cls_folder in os.listdir(data_path):
    os.makedirs(out_foler / cls_folder, exist_ok=True)
    for filename in tqdm(os.listdir(data_path / cls_folder / 'images')):
        if os.path.exists(str(out_foler / cls_folder / filename)):
            continue

        img = Image.open(str(data_path / cls_folder / 'images' / filename))
        results = model(img, verbose=False)

        img = cv2.cvtColor(np.asarray(img), cv2.COLOR_BGR2RGB)
        if img is None:
            continue

        cls = results[0].probs
        if results[0].masks is None:
            continue
        masks = results[0].masks.xy
        for swan_contours in masks:
            swan_mask = np.zeros_like(img)
            cv2.drawContours(swan_mask, [swan_contours.astype(np.int32)], 0, (255,255,255), -1)

            swan_mask = swan_mask[:, :, 0]
            contours, hierarchy = cv2.findContours(swan_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

            left = min([min(cnt[:, 0, 0]) for cnt in contours])
            right = max([max(cnt[:, 0, 0]) for cnt in contours])

            top = min([min(cnt[:, 0, 1]) for cnt in contours])
            bottom = max([max(cnt[:, 0, 1]) for cnt in contours])

            swan = deepcopy(img)[top:bottom, left:right]
            swan_mask = swan_mask[top:bottom, left:right]
            
            swan = cv2.bitwise_and(swan, swan, mask=swan_mask)
            cv2.imwrite(str(out_foler / cls_folder.replace('разметка_малый', 'malyi').replace('разметка_шипун', 'shipun') / filename), swan)

100%|█████████▉| 3011/3012 [06:23<00:00,  7.86it/s]


PermissionError: [Errno 13] Permission denied: 'razmetka\\разметка_шипун\\images\\masks'

In [12]:
data_path = Path(f'./razmetka')
out_foler = Path(f'./yolo_preds_head')
for cls_folder in os.listdir(data_path):
    os.makedirs(out_foler / cls_folder, exist_ok=True)
    for filename in tqdm(os.listdir(data_path / cls_folder / 'images')):
        if os.path.exists(str(out_foler / cls_folder / filename)):
            continue

        img = Image.open(str(data_path / cls_folder / 'images' / filename))
        results = model(img, verbose=False)

        img = cv2.cvtColor(np.asarray(img), cv2.COLOR_BGR2RGB)
        if img is None:
            continue

        cls = results[0].probs
        if results[0].boxes is None:
            continue
        boxes = results[0].boxes.xyxy
        for box in boxes:
            box = box.cpu().detach().numpy().astype(np.int32)

            swan = deepcopy(img)[box[1]:box[3], box[0]:box[2]]

            cv2.imwrite(str(out_foler / cls_folder.replace('разметка_малый', 'malyi').replace('разметка_шипун', 'shipun') / filename), swan)

100%|█████████▉| 3011/3012 [05:42<00:00,  8.79it/s]


PermissionError: [Errno 13] Permission denied: 'razmetka\\разметка_шипун\\images\\masks'

In [9]:
box

tensor([ 666.5624,  339.6073, 1001.2791,  518.9988], device='cuda:0')

In [85]:
import timm

model = timm.create_model('../min_nature/min_nature/weights/convnext/', pretrained=True)

RuntimeError: Unknown model ()

In [87]:
from urllib.request import urlopen
from PIL import Image
import timm
from transformers import AutoModel, AutoProcessor


model = AutoModel.from_pretrained('../min_nature/weights/convnext/').to('cuda')
model = model.eval()

data_config = timm.data.resolve_model_data_config(model)
transforms = timm.data.create_transform(**data_config, is_training=False)

Some weights of the model checkpoint at ../min_nature/weights/convnext/ were not used when initializing ConvNextModel: ['stages.3.blocks.1.mlp.fc1.weight', 'stages.2.blocks.17.mlp.fc1.bias', 'stages.2.blocks.10.norm.weight', 'head.norm.weight', 'stages.2.blocks.29.conv_dw.bias', 'stages.2.blocks.22.weight', 'stages.2.blocks.14.weight', 'stages.2.blocks.21.conv_dw.bias', 'stages.0.blocks.0.norm.weight', 'stages.2.blocks.0.mlp.fc2.bias', 'stages.3.downsample.1.bias', 'stages.2.blocks.18.weight', 'stages.2.blocks.23.norm.bias', 'stages.2.blocks.2.weight', 'stages.1.blocks.3.weight', 'stages.2.blocks.13.conv_dw.bias', 'stages.2.blocks.26.norm.bias', 'stages.2.blocks.15.mlp.fc1.bias', 'stages.2.blocks.11.conv_dw.weight', 'stages.2.blocks.15.conv_dw.weight', 'stages.2.blocks.19.mlp.fc1.bias', 'stages.2.blocks.9.mlp.fc1.bias', 'stages.2.blocks.24.norm.bias', 'stages.2.blocks.2.mlp.fc1.bias', 'head.fc.bias', 'stages.2.blocks.23.mlp.fc2.weight', 'stages.2.blocks.20.weight', 'stages.2.blocks.28.

In [ ]:
data_path = Path(f'./razmetka')
for cls_folder in os.listdir(data_path):
    os.makedirs(out_foler / cls_folder, exist_ok=True)
    for filename in tqdm(os.listdir(data_path / cls_folder / 'images')):
        img = Image.open(str(data_path / cls_folder / 'images' / filename))
        

In [88]:
img = Image.open(urlopen(
    'https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/beignets-task-guide.png'
))

In [91]:
model(img)

AttributeError: shape

In [95]:
output = model(transforms(img).unsqueeze(0).to('cuda'))  # output is (batch_size, num_features) shaped tensor

# or equivalently (without needing to set num_classes=0)

output = model.forward_features(transforms(img).unsqueeze(0))
# output is unpooled, a (1, 3072, 8, 8) shaped tensor

output = model.forward_head(output, pre_logits=True)

AttributeError: 'ConvNextModel' object has no attribute 'forward_features'

In [104]:
model

ConvNextModel(
  (embeddings): ConvNextEmbeddings(
    (patch_embeddings): Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4))
    (layernorm): ConvNextLayerNorm()
  )
  (encoder): ConvNextEncoder(
    (stages): ModuleList(
      (0): ConvNextStage(
        (downsampling_layer): Identity()
        (layers): Sequential(
          (0): ConvNextLayer(
            (dwconv): Conv2d(96, 96, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=96)
            (layernorm): ConvNextLayerNorm()
            (pwconv1): Linear(in_features=96, out_features=384, bias=True)
            (act): GELUActivation()
            (pwconv2): Linear(in_features=384, out_features=96, bias=True)
            (drop_path): Identity()
          )
          (1): ConvNextLayer(
            (dwconv): Conv2d(96, 96, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=96)
            (layernorm): ConvNextLayerNorm()
            (pwconv1): Linear(in_features=96, out_features=384, bias=True)
            (act): GELUAc

In [109]:
model.encoder(transforms(img).unsqueeze(0).to('cuda'))

RuntimeError: Given groups=96, weight of size [96, 1, 7, 7], expected input[1, 3, 224, 224] to have 96 channels, but got 3 channels instead

In [108]:
model.encoder

ConvNextEncoder(
  (stages): ModuleList(
    (0): ConvNextStage(
      (downsampling_layer): Identity()
      (layers): Sequential(
        (0): ConvNextLayer(
          (dwconv): Conv2d(96, 96, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=96)
          (layernorm): ConvNextLayerNorm()
          (pwconv1): Linear(in_features=96, out_features=384, bias=True)
          (act): GELUActivation()
          (pwconv2): Linear(in_features=384, out_features=96, bias=True)
          (drop_path): Identity()
        )
        (1): ConvNextLayer(
          (dwconv): Conv2d(96, 96, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=96)
          (layernorm): ConvNextLayerNorm()
          (pwconv1): Linear(in_features=96, out_features=384, bias=True)
          (act): GELUActivation()
          (pwconv2): Linear(in_features=384, out_features=96, bias=True)
          (drop_path): Identity()
        )
        (2): ConvNextLayer(
          (dwconv): Conv2d(96, 96, kernel_size=(7, 7), s

In [107]:
dir(model)

['T_destination',
 '__annotations__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_apply',
 '_auto_class',
 '_backward_compatibility_gradient_checkpointing',
 '_backward_hooks',
 '_backward_pre_hooks',
 '_buffers',
 '_call_impl',
 '_can_retrieve_inputs_from_name',
 '_convert_head_mask_to_5d',
 '_create_repo',
 '_expand_inputs_for_generation',
 '_forward_hooks',
 '_forward_hooks_with_kwargs',
 '_forward_pre_hooks',
 '_forward_pre_hooks_with_kwargs',
 '_from_config',
 '_get_backward_hooks',
 '_get_backward_pre_hooks',
 '_get_decoder_start_token_id',
 '_get_files_timestamps',
 '_get_logits_processor',
 '_get_logits_warper

In [103]:
output.last_hidden_state.shape

torch.Size([1, 768, 7, 7])

In [ ]:




# get model specific transforms (normalization, resize)


output = model(transforms(img).unsqueeze(0).to('cuda'))  # output is (batch_size, num_features) shaped tensor

# or equivalently (without needing to set num_classes=0)

output = model.forward_features(transforms(img).unsqueeze(0))
# output is unpooled, a (1, 3072, 8, 8) shaped tensor

output = model.forward_head(output, pre_logits=True)

In [74]:
img

array([[[ 74,  85,  87],
        [ 73,  84,  86],
        [ 72,  83,  85],
        ...,
        [ 85,  96, 102],
        [ 87,  98, 104],
        [ 89, 100, 106]],

       [[ 77,  88,  90],
        [ 76,  87,  89],
        [ 75,  86,  88],
        ...,
        [ 90, 101, 107],
        [ 92, 103, 109],
        [ 94, 105, 111]],

       [[ 74,  85,  87],
        [ 73,  84,  86],
        [ 72,  83,  85],
        ...,
        [ 88,  99, 105],
        [ 90, 101, 107],
        [ 92, 103, 109]],

       ...,

       [[ 93, 104, 106],
        [103, 114, 116],
        [ 99, 110, 112],
        ...,
        [118, 124, 124],
        [121, 127, 127],
        [124, 130, 130]],

       [[ 94, 105, 107],
        [ 97, 108, 110],
        [ 98, 109, 111],
        ...,
        [119, 125, 125],
        [123, 129, 129],
        [126, 132, 132]],

       [[ 96, 107, 109],
        [ 99, 110, 112],
        [100, 111, 113],
        ...,
        [122, 128, 128],
        [125, 131, 131],
        [128, 134, 134]]

In [72]:
masks

[array([[      472.5,       202.5],
        [     470.62,      204.37],
        [     468.75,      204.37],
        ...,
        [     545.62,      204.37],
        [     543.75,      204.37],
        [     541.88,       202.5]], dtype=float32),
 array([[      9.375,       562.5],
        [        7.5,      564.38],
        [        7.5,      639.37],
        [      9.375,      641.25],
        [      11.25,      641.25],
        [     13.125,      643.12],
        [      41.25,      643.12],
        [     43.125,      641.25],
        [       52.5,      641.25],
        [     54.375,      639.37],
        [      56.25,      639.37],
        [     58.125,       637.5],
        [         60,       637.5],
        [     65.625,      631.87],
        [     65.625,      616.87],
        [       67.5,         615],
        [       67.5,      605.62],
        [     65.625,      603.75],
        [     65.625,      594.37],
        [      63.75,       592.5],
        [     61.875,       592.5]

In [65]:
img = cv2.imread(str(data_path / cls_folder / 'images' / filename))

In [67]:
str(data_path / cls_folder / 'images' / filename)

'razmetka\\разметка_малый\\images\\00120000501.jpg'

In [66]:
img

In [61]:
swan_mask.astype(np.uint8)

array([[        736,       349.7],
       [      723.2,       362.5],
       [        720,       362.5],
       [      716.8,       365.7],
       [      716.8,       368.9],
       [      710.4,       375.3],
       [      707.2,       375.3],
       [        704,       378.5],
       [        704,       381.7],
       [      691.2,       394.5],
       [      691.2,       404.1],
       [      678.4,       416.9],
       [      678.4,       471.3],
       [      675.2,       474.5],
       [      675.2,       484.1],
       [        672,       487.3],
       [        672,       493.7],
       [      668.8,       496.9],
       [      668.8,       506.5],
       [      665.6,       509.7],
       [      665.6,       704.9],
       [      662.4,       708.1],
       [      662.4,       717.7],
       [      659.2,       720.9],
       [      659.2,       727.3],
       [        656,       730.5],
       [        656,       756.1],
       [      652.8,       759.3],
       [      652.8,

In [57]:
masks

[array([[      472.5,       202.5],
        [     470.62,      204.37],
        [     468.75,      204.37],
        ...,
        [     545.62,      204.37],
        [     543.75,      204.37],
        [     541.88,       202.5]], dtype=float32),
 array([[      9.375,       562.5],
        [        7.5,      564.38],
        [        7.5,      639.37],
        [      9.375,      641.25],
        [      11.25,      641.25],
        [     13.125,      643.12],
        [      41.25,      643.12],
        [     43.125,      641.25],
        [       52.5,      641.25],
        [     54.375,      639.37],
        [      56.25,      639.37],
        [     58.125,       637.5],
        [         60,       637.5],
        [     65.625,      631.87],
        [     65.625,      616.87],
        [       67.5,         615],
        [       67.5,      605.62],
        [     65.625,      603.75],
        [     65.625,      594.37],
        [      63.75,       592.5],
        [     61.875,       592.5]

In [45]:
results[0].names

{0: 'mute', 1: 'whooper', 2: 'bewick'}

In [55]:
results[0].boxes[1].cls

tensor([1.], device='cuda:0')

In [53]:
type(results[0].boxes.data)

torch.Tensor

In [38]:
dir(model)

['__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_check_is_pytorch_model',
 '_load',
 '_new',
 '_reset_callbacks',
 '_reset_ckpt_args',
 'add_callback',
 'benchmark',
 'callbacks',
 'cfg',
 'ckpt',
 'ckpt_path',
 'clear_callback',
 'device',
 'export',
 'fuse',
 'info',
 'is_hub_model',
 'load',
 'metrics',
 'model',
 'names',
 'overrides',
 'predict',
 'predictor',
 'reset_weights',
 'session',
 'task',
 'to',
 'track',
 'train',
 'trainer',
 'transforms',
 'tune',
 'val']

In [27]:
masks

[array([[        736,       349.7],
        [      723.2,       362.5],
        [        720,       362.5],
        [      716.8,       365.7],
        [      716.8,       368.9],
        [      710.4,       375.3],
        [      707.2,       375.3],
        [        704,       378.5],
        [        704,       381.7],
        [      691.2,       394.5],
        [      691.2,       404.1],
        [      678.4,       416.9],
        [      678.4,       471.3],
        [      675.2,       474.5],
        [      675.2,       484.1],
        [        672,       487.3],
        [        672,       493.7],
        [      668.8,       496.9],
        [      668.8,       506.5],
        [      665.6,       509.7],
        [      665.6,       704.9],
        [      662.4,       708.1],
        [      662.4,       717.7],
        [      659.2,       720.9],
        [      659.2,       727.3],
        [        656,       730.5],
        [        656,       756.1],
        [      652.8,       

In [26]:
results[0].masks.data

tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        ...,

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0., 

In [23]:
results[0]

ultralytics.yolo.engine.results.Results object with attributes:

boxes: ultralytics.yolo.engine.results.Boxes object
keypoints: None
keys: ['boxes', 'masks']
masks: ultralytics.yolo.engine.results.Masks object
names: {0: 'mute', 1: 'whooper', 2: 'bewick'}
orig_img: array([[[186, 159, 138],
        [186, 159, 139],
        [186, 158, 141],
        ...,
        [182, 155, 135],
        [181, 154, 134],
        [184, 154, 135]],

       [[187, 161, 137],
        [186, 159, 138],
        [186, 159, 139],
        ...,
        [182, 155, 135],
        [182, 154, 137],
        [182, 153, 138]],

       [[186, 159, 138],
        [185, 158, 137],
        [185, 158, 137],
        ...,
        [181, 154, 134],
        [181, 154, 134],
        [181, 153, 136]],

       ...,

       [[180, 181, 177],
        [176, 179, 177],
        [177, 180, 178],
        ...,
        [ 28,  28,  22],
        [ 28,  25,  20],
        [ 32,  29,  24]],

       [[178, 179, 177],
        [178, 179, 177],
        [17

In [20]:
cls

In [15]:
len(masks)

15

In [18]:
masks[1
]

array([[        736,       349.7],
       [      723.2,       362.5],
       [        720,       362.5],
       [      716.8,       365.7],
       [      716.8,       368.9],
       [      710.4,       375.3],
       [      707.2,       375.3],
       [        704,       378.5],
       [        704,       381.7],
       [      691.2,       394.5],
       [      691.2,       404.1],
       [      678.4,       416.9],
       [      678.4,       471.3],
       [      675.2,       474.5],
       [      675.2,       484.1],
       [        672,       487.3],
       [        672,       493.7],
       [      668.8,       496.9],
       [      668.8,       506.5],
       [      665.6,       509.7],
       [      665.6,       704.9],
       [      662.4,       708.1],
       [      662.4,       717.7],
       [      659.2,       720.9],
       [      659.2,       727.3],
       [        656,       730.5],
       [        656,       756.1],
       [      652.8,       759.3],
       [      652.8,